In [ ]:
import importlib
import numpy as np
import matplotlib.pyplot as plt

import colors, plot_func, helpers, file_helpers, experiments

## Supervisor

In [ ]:
import SupervisorClass
importlib.reload(SupervisorClass)
from SupervisorClass import SupervisorClass

Sprvsr = SupervisorClass()
# Sprvsr.init_dataset()

In [ ]:
importlib.reload(file_helpers)
importlib.reload(plot_func)

import ForsentekClass
importlib.reload(ForsentekClass)
from ForsentekClass import ForsentekClass

Snsr = ForsentekClass()

In [ ]:
import MecaClass
importlib.reload(MecaClass)
from MecaClass import MecaClass

m = MecaClass()

In [ ]:
m.connect()

In [ ]:
m.robot.GetPose()
# m.move_joints((10, 56.5, 0, 0, 30, 180))
m.move_to_origin()
Snsr.calibrate_daily(V0_from_file = False)

## Check Robot movement

In [ ]:
# m.robot.MoveLin(140, 0, 180, 180, 0, 0)
# points = (220.0, 0.0, 360, 0, 0, 90)
points = (240, 80, 45)
m.move_pos(points)
# joints = (0, 60, 30, -90, -90, 180)
# m.move_joints(joints)

In [ ]:
Snsr.measure(4)
Sprvsr.global_force(Snsr, m, plot=True)
print('Fx', Sprvsr.Fx)
print('Fy', Sprvsr.Fy)

## Build dataset

In [ ]:
importlib.reload(experiments)

x_range = 240
y_range = 85
theta_range = 45
N = 8
x_y_theta_vec, F_vec = experiments.sweep_measurement_fixed_origami(m, Snsr, Sprvsr, 240, 85, 45, N)
print('x_y_theta_vec', x_y_theta_vec)
print('F_vec', F_vec)

In [ ]:
plt.plot(x_y_theta_vec[:,1], F_vec[0,:])
plt.plot(x_y_theta_vec[:,1], F_vec[1,:])
plt.show()

In [ ]:
origin = np.array([154-30, -14, 0])
x_y_theta_vec_moved = x_y_theta_vec - origin
print(x_y_theta_vec_moved)
file_helpers.write_supervisor_dataset(x_y_theta_vec_moved, F_vec, out_path="buckle[1, -1, -1, -1, -1].csv")

In [ ]:
importlib.reload(file_helpers)

out_path = 'buckle=[1, 1, -1, -1, 1].csv'
file_helpers.write_supervisor_dataset(x_y_theta_vec, F_vec,
                                      out_path=out_path)

## Training

In [ ]:
Sprvsr.init_dataset(dataset_path = "data/datasets/buckle=[1, 1, -1, -1, 1].csv")
Sprvsr.pos_in_t

In [ ]:
for t in range(Sprvsr.T):

    # # measurement
    print('tip position = ', Sprvsr.pos_in_t[t, :])
    m.move_pos(Sprvsr.pos_in_t[t, :])
    print('measuring force')
    Snsr.measure()
    Sprvsr.global_force(Snsr, m, t, plot=True)
    print('saving mean force = ', [Sprvsr.Fx, Sprvsr.Fy])
    Sprvsr.calc_loss(t, Snsr.norm_force)
    print('loss (normalized) = ', Sprvsr.loss_norm)

    # # update
    Sprvsr.calc_tip_update(m, t)
    m.move_pos(Sprvsr.pos_update_in_t[t, :])

In [ ]:
print('update pos in t', Sprvsr.pos_update_in_t)
print('Sprvsr.F_in_t', Sprvsr.F_in_t)
print('Sprvsr.desired_F_in_t', Sprvsr.desired_F_in_t)
plt.plot(range(Sprvsr.T), Sprvsr.loss_norm_in_t)

In [ ]:
# Sprvsr.pos_in_t = np.array([[180, 40, 0], [180, 40, -40], [180, 40, -80], 
#                             [180, 40, -120], [180, 40, -140], [180, 40, -160], 
#                             [180, 40, -180], [180, 40, -200] ])
# Sprvsr.pos_in_t = np.array([[180, 70, -90]])
# Sprvsr.pos_in_t = np.array([[180, 70, -90], [180, 75, -90], [180, 80, -90], 
#                             [180, 85, -90], [180, 90, -90], [180, 95, -90], 
#                             [180, 100, -90], [180, 105, -90]])
# Sprvsr.T = 8
# tri = 1 - np.abs(2 * (np.arange(Sprvsr.T) / (Sprvsr.T - 1)) - 1)   # 0 → 1 → 0
# x_pos = 230 + 10 * tri                   # 170 → 210 → 170
# y_pos = np.linspace(-50, 50, Sprvsr.T)
# Sprvsr.pos_in_t = np.zeros([Sprvsr.T, 3])
# Sprvsr.F_in_t = np.zeros([Sprvsr.T, 2])

# for t in range(Sprvsr.T):
#     Sprvsr.pos = np.array([x_pos[t], y_pos[t], 0])
#     Sprvsr.pos_in_t[t, :] = Sprvsr.pos
#     print('tip position = ', Sprvsr.pos)
#     m.move_pos(Sprvsr.pos)
#     Snsr.measure()
#     Sprvsr.global_force(Snsr, m, plot=True)
#     plt.show()
#     Sprvsr.F_in_t[t, :] = np.array([Sprvsr.Fx, Sprvsr.Fy])
#     print('mean force = ', [Sprvsr.Fx, Sprvsr.Fy])

In [ ]:
plt.errorbar(y_pos, Sprvsr.F_in_t[:,0], yerr=0.01)
plt.errorbar(y_pos, Sprvsr.F_in_t[:,1], yerr=0.01)
plt.ylabel('F [N]')
plt.ylim([-0.015, 0.085])
plt.xlabel('y [mm]')
plt.legend(['Fx', 'Fy'])

In [ ]:
m.disconnect()

## Logging - fixed arm

In [ ]:
experiments.sweep_measurement_fixed_origami(x_range, y_range, theta_range, N, robot, force_sensor)

## Calibrate forces

In [ ]:
importlib.reload(experiments)

weights = [5, 20]
voltages_arr, forces_arr, stds_arr, fit_params_arr = experiments.calibrate_forces_all_axes(m, Snsr, weights)

In [ ]:
fit_params_arr

In [ ]:
from numpy.polynomial import polynomial as poly
dim=0
poly.polyfit(voltages_arr[:, dim], forces_arr[:, dim], deg=1)

In [ ]:
forces_arr[:, dim]

## Recover robot after error

In [ ]:
m.recover_robot()

## End of day

In [ ]:
# m.robot.MoveLin(40, 0, 230, 180, 0, 0)
m.move_to_sleep_pos()

In [ ]:
m.disconnect()